# CelcomDigi Snowflake Intelligence - Hands-On Lab

## Learning Objectives

This hands-on lab teaches you to analyze telecommunications data using Snowflake Cortex AI functions. You will:

1. Query and explore structured and unstructured data
2. Use Cortex AISQL functions for text analysis and translation
3. Combine customer, network, and call data for insights
4. Create visualizations to identify business opportunities
5. Leverage Cortex Search for semantic queries
6. Calculate business metrics and risk indicators

**Prerequisites:** 
- data_processing.ipynb has been run successfully
- **Important:** Add `matplotlib` to notebook packages
  - In Snowflake notebook settings
  - Click "Packages" dropdown
  - Type "matplotlib" and add it
  - This enables all visualizations in this lab

**Duration:** 45-60 minutes


In [ ]:
# Setup
from snowflake.snowpark import Session
import pandas as pd
import matplotlib.pyplot as plt

session = Session.builder.getOrCreate()
print(f"Connected: {session.get_current_database()}.{session.get_current_schema()}")


## Exercise 1: Data Exploration and Summary Statistics

**What you'll learn:**
- Query multiple tables to understand data volume
- View sample records from processed transcripts
- Verify data loading was successful

**Business value:** Understand the scope of available data for analysis


In [ ]:
# Check all tables
tables = ['network_performance', 'customer_details', 'customer_call_transcripts', 
          'customer_complaint_documents', 'csat_surveys']

for table in tables:
    count = session.table(table).count()
    print(f"{table:40s}: {count:>6,} records")

# Sample calls
print("\\nSample Calls:")
session.table("customer_call_transcripts").select("call_id", "call_reason", "summary").show(5)


## Exercise 2: Multilingual Translation with AI_TRANSLATE

**What you'll learn:**
- Use AI_TRANSLATE to convert text between languages
- Support multilingual teams with automatic translation
- Understand supported language codes

**Business value:** Enable global teams to analyze customer feedback in their preferred language


In [ ]:
# Translate summaries to Chinese and Japanese (for regional teams)
# Note: AI_TRANSLATE supports: en, zh, ja, ko, es, fr, de, it, pt, etc.
sample = session.sql("SELECT summary FROM customer_call_transcripts LIMIT 2").collect()

print("Multi-Language Translation Example:\\n")
for row in sample:
    summary = row['SUMMARY']
    print(f"English: {summary}")
    
    # Translate to Chinese
    zh = session.sql(f"SELECT SNOWFLAKE.CORTEX.AI_TRANSLATE('{summary}', 'en', 'zh')").collect()[0][0]
    print(f"Chinese: {zh}")
    
    # Translate to Japanese
    ja = session.sql(f"SELECT SNOWFLAKE.CORTEX.AI_TRANSLATE('{summary}', 'en', 'ja')").collect()[0][0]
    print(f"Japanese: {ja}\\n")


## Exercise 3: Customer Sentiment Analysis with Visualization

**What you'll learn:**
- Extract sentiment from AI_SENTIMENT VARIANT results
- Aggregate sentiment across all customer calls
- Create bar charts to visualize sentiment distribution

**Business value:** Identify patterns in customer emotions and satisfaction levels


In [ ]:
# Sentiment distribution
sentiment = session.sql("""
SELECT 
    sentiment_score:categories[0]:sentiment::VARCHAR as sentiment,
    COUNT(*) as count
FROM customer_call_transcripts
GROUP BY sentiment
""").to_pandas()

print(sentiment)

# Chart (column names are uppercase from Snowflake)
colors = {'positive': 'green', 'negative': 'red', 'neutral': 'gray', 'mixed': 'orange'}
plt.figure(figsize=(8, 5))
plt.bar(sentiment['SENTIMENT'], sentiment['COUNT'], 
        color=[colors.get(s, 'blue') for s in sentiment['SENTIMENT']])
plt.ylabel('Number of Calls')
plt.title('Sentiment Distribution')
plt.show()


## Exercise 4: Customer Satisfaction (CSAT) Analysis

**What you'll learn:**
- Analyze CSAT score distribution across customer surveys
- Calculate average satisfaction metrics
- Create histograms to identify satisfaction trends

**Business value:** Measure customer satisfaction and track service quality performance


In [ ]:
# CSAT distribution
csat = session.sql("""
SELECT csat_score, COUNT(*) as count
FROM csat_surveys
GROUP BY csat_score
ORDER BY csat_score
""").to_pandas()

plt.figure(figsize=(8, 5))
plt.bar(csat['CSAT_SCORE'], csat['COUNT'], color='teal')
plt.xlabel('CSAT Score (1-5)')
plt.ylabel('Count')
plt.title('CSAT Distribution')
plt.xticks([1, 2, 3, 4, 5])
plt.show()

avg = (csat['CSAT_SCORE'] * csat['COUNT']).sum() / csat['COUNT'].sum()
print(f"Average CSAT: {avg:.2f}")


## Exercise 5: Network Performance Analysis by Region

**What you'll learn:**
- Aggregate network metrics by geographic region
- Compare performance across Malaysian states
- Identify regions with network quality issues

**Business value:** Prioritize infrastructure investments based on regional performance


In [ ]:
# Performance by region (use all available data)
perf = session.sql("""
SELECT region, 
       AVG(avg_latency_ms) as latency,
       AVG(packet_loss_pct) as packet_loss
FROM network_performance
GROUP BY region
ORDER BY latency DESC
""").to_pandas()

print("Network Performance by Region:")
print(perf)

if len(perf) > 0:
    plt.figure(figsize=(10, 5))
    plt.barh(perf['REGION'], perf['LATENCY'])
    plt.xlabel('Average Latency (ms)')
    plt.title('Network Latency by Region')
    plt.tight_layout()
    plt.show()
else:
    print("No network performance data available")


## Exercise 6: Semantic Search with Cortex Search

**What you'll learn:**
- Perform semantic search across call transcripts and PDF documents
- Use natural language queries to find relevant information
- Access both structured and unstructured data simultaneously

**Business value:** Quickly find relevant customer interactions and documentation without exact keyword matching

**Note:** If search service doesn't exist, verify:
- `SHOW CORTEX SEARCH SERVICES;` in SQL
- Re-run the Cortex Search creation section from setup.sql if needed


In [ ]:
# Search across transcripts and PDFs
try:
    results = session.sql("""
    SELECT FEEDBACK_ID, SOURCE, SENTIMENT
    FROM TABLE(celcomdigi_feedback_search!SEARCH('network problems', 5))
    """).to_pandas()
    print(f"Found {len(results)} results:")
    print(results[['FEEDBACK_ID', 'SOURCE', 'SENTIMENT']])
except Exception as e:
    print(f"Search not ready yet: {e}")


## Exercise 7: Customer Risk Identification and Segmentation

**What you'll learn:**
- Use customer_360_view to identify at-risk customers
- Segment customers by risk level
- Visualize risk distribution across customer segments

**Business value:** Proactively identify customers likely to churn for retention interventions


In [ ]:
# Find at-risk customers
risk = session.sql("""
SELECT customer_segment, 
       COUNT(*) as total,
       SUM(CASE WHEN is_at_risk THEN 1 ELSE 0 END) as at_risk
FROM customer_360_view
GROUP BY customer_segment
""").to_pandas()

print(risk)

plt.figure(figsize=(10, 5))
x = range(len(risk))
plt.bar(x, risk['TOTAL'], label='Total', alpha=0.6)
plt.bar(x, risk['AT_RISK'], label='At Risk', color='red')
plt.xticks(x, risk['CUSTOMER_SEGMENT'])
plt.ylabel('Customers')
plt.title('Customer Risk by Segment')
plt.legend()
plt.show()


## Exercise 8: Revenue Impact and Risk Quantification

**What you'll learn:**
- Calculate revenue at risk from dissatisfied customers
- Aggregate financial impact by customer segment
- Visualize revenue exposure from potential churn

**Business value:** Quantify financial impact of customer dissatisfaction to justify retention investments


In [ ]:
# Revenue at risk
revenue = session.sql("""
SELECT customer_segment,
       SUM(CASE WHEN is_at_risk THEN monthly_revenue ELSE 0 END) as revenue_at_risk
FROM customer_360_view
GROUP BY customer_segment
ORDER BY revenue_at_risk DESC
""").to_pandas()

print(revenue)

plt.figure(figsize=(8, 5))
plt.bar(revenue['CUSTOMER_SEGMENT'], revenue['REVENUE_AT_RISK'], color='darkred')
plt.ylabel('Monthly Revenue at Risk (RM)')
plt.title('Revenue at Risk by Segment')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"\\nTotal at Risk: RM {revenue['REVENUE_AT_RISK'].sum():,.2f}")


## Exercise 9: Data Privacy with AI_REDACT

**What you'll learn:**
- Use AI_REDACT to automatically remove PII from text
- Protect customer privacy while enabling data analysis
- Create anonymized datasets for sharing

**Business value:** Comply with data privacy regulations while maintaining analytical capabilities


In [ ]:
# Redact PII from transcript
sample = session.sql("SELECT transcript_text FROM customer_call_transcripts LIMIT 1").collect()

if len(sample) > 0:
    text = sample[0]['TRANSCRIPT_TEXT'][:500]
    print("Original:\\n", text)
    
    redacted = session.sql(f"SELECT SNOWFLAKE.CORTEX.AI_REDACT('{text}')").collect()[0][0]
    print("\\nRedacted:\\n", redacted)
    print("\\nUse case: Share data while protecting customer privacy")


## Exercise 10: Custom Analysis and Experimentation

**What you'll learn:**
- Apply learned concepts to create custom analyses
- Combine multiple AI functions for complex insights
- Develop your own SQL queries and visualizations

**Business value:** Build custom analytics tailored to specific business questions

**Try this:**
- Analyze competitor mentions and customer sentiment
- Create additional visualizations
- Experiment with other Cortex AI functions (AI_COMPLETE, AI_EXTRACT, etc.)


In [ ]:
# Example: Calls mentioning competitors
competitor = session.sql("""
SELECT call_id, call_reason,
       sentiment_score:categories[0]:sentiment::VARCHAR as sentiment,
       summary
FROM customer_call_transcripts
WHERE LOWER(transcript_text) LIKE '%maxis%' 
   OR LOWER(transcript_text) LIKE '%mobile%'
LIMIT 5
""").to_pandas()

print("Calls Mentioning Competitors:")
print(competitor)

# Your custom analysis here:
# - Try different SQL queries
# - Use other AI functions (AI_COMPLETE, AI_CLASSIFY)
# - Create your own visualizations

print("\\nLab Complete! Now use Snowflake Intelligence Agent for natural language queries.")


## Exercise 11: ML Prediction - Churn Risk Scoring Model

**What you'll learn:**
- Build a weighted scoring model to predict customer churn
- Use multiple features (CSAT, complaints, unresolved issues) for prediction
- Compare predicted vs actual churn rates
- Visualize model performance with 4 comprehensive charts

**Business objectives:**
- **Proactive Retention:** Identify at-risk customers before they churn to competitors (Maxis, U Mobile)
- **Resource Optimization:** Prioritize retention efforts on high-risk, high-value customers
- **Revenue Protection:** Quantify and prevent revenue loss from customer churn
- **Operational Efficiency:** Automate churn risk assessment instead of manual analysis

**Model features and weights:**
- Low CSAT Score (<3): 30 points - Strong predictor of dissatisfaction
- High Complaints (>2): 25 points - Indicates ongoing service issues
- Unresolved Issues (>0): 25 points - Critical driver of churn
- Frequent Calls (>2): 20 points - Sign of persistent problems

**Risk threshold:** Score >50 indicates high churn probability

**Expected outcome:** Actionable list of customers requiring immediate retention intervention


In [ ]:
# Simple churn prediction model
churn = session.sql("""
SELECT 
    customer_id,
    customer_segment,
    total_calls,
    total_complaints,
    avg_csat_score,
    unresolved_issues,
    is_at_risk,
    is_churned
FROM customer_360_view c
JOIN customer_details d USING (customer_id)
WHERE total_calls > 0
""").to_pandas()

print(f"Dataset: {len(churn)} customers\\n")

# Calculate churn risk score
churn['risk_score'] = (
    (churn['AVG_CSAT_SCORE'] < 3).astype(int) * 30 +  # Low satisfaction
    (churn['TOTAL_COMPLAINTS'] > 2).astype(int) * 25 +  # Multiple complaints
    (churn['UNRESOLVED_ISSUES'] > 0).astype(int) * 25 +  # Unresolved issues
    (churn['TOTAL_CALLS'] > 2).astype(int) * 20  # Frequent caller
)

churn['predicted_churn'] = churn['risk_score'] > 50

# Results
print("Prediction Results:")
print(f"Predicted High Risk: {churn['predicted_churn'].sum()}")
print(f"Actual Churned: {churn['IS_CHURNED'].sum()}")

# Visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))

# 1. Risk Score Distribution
ax1.hist(churn['risk_score'], bins=15, color='coral', edgecolor='black', alpha=0.7)
ax1.axvline(x=50, color='red', linestyle='--', linewidth=2, label='Churn Threshold')
ax1.set_xlabel('Churn Risk Score')
ax1.set_ylabel('Number of Customers')
ax1.set_title('Churn Risk Score Distribution', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Feature Importance
features = ['Low CSAT', 'High Complaints', 'Unresolved Issues', 'Frequent Caller']
weights = [30, 25, 25, 20]
ax2.barh(features, weights, color=['red', 'orange', 'coral', 'yellow'])
ax2.set_xlabel('Weight in Risk Score')
ax2.set_title('Churn Prediction Features', fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

# 3. CSAT vs Risk Score
colors = churn['IS_CHURNED'].map({True: 'red', False: 'green'})
ax3.scatter(churn['AVG_CSAT_SCORE'], churn['risk_score'], c=colors, alpha=0.6, s=100)
ax3.set_xlabel('Average CSAT Score')
ax3.set_ylabel('Churn Risk Score')
ax3.set_title('CSAT vs Churn Risk (Red=Churned, Green=Active)', fontweight='bold')
ax3.axhline(y=50, color='red', linestyle='--', alpha=0.5)
ax3.axvline(x=3, color='orange', linestyle='--', alpha=0.5)
ax3.grid(True, alpha=0.3)

# 4. Predictions by Segment
seg = churn.groupby('CUSTOMER_SEGMENT').agg({
    'predicted_churn': 'sum',
    'IS_CHURNED': 'sum'
})
x = range(len(seg))
width = 0.35
ax4.bar([i-width/2 for i in x], seg['predicted_churn'], width, label='Predicted', color='orange', alpha=0.8)
ax4.bar([i+width/2 for i in x], seg['IS_CHURNED'], width, label='Actual', color='red', alpha=0.8)
ax4.set_xticks(x)
ax4.set_xticklabels(seg.index, rotation=45, ha='right')
ax4.set_ylabel('Number of Customers')
ax4.set_title('Churn: Predicted vs Actual by Segment', fontweight='bold')
ax4.legend()
ax4.grid(axis='y', alpha=0.3)

plt.suptitle('Churn Prediction Model', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\\nUse this model to:")
print("- Identify at-risk customers before they churn")
print("- Prioritize retention efforts by risk score")
print("- Target high-value customers with proactive support")


## Lab Summary and Completion

### What You Accomplished

Congratulations! You have completed all 11 exercises and learned to:

**Cortex AI Functions:**
- AI_TRANSLATE for multilingual support
- AI_REDACT for data privacy compliance
- AI_SENTIMENT for emotion analysis (VARIANT handling)
- Cortex Search for semantic queries across structured and unstructured data

**Data Analysis:**
- Explored 81,425+ records across multiple tables
- Analyzed customer call transcripts and PDF documents
- Calculated CSAT scores and sentiment patterns
- Identified network performance issues by region

**Business Intelligence:**
- Identified at-risk customers using customer_360_view
- Quantified revenue at risk from potential churn
- Built churn prediction model with feature scoring
- Created visualizations for executive dashboards

### Key Takeaways

1. **Structured + Unstructured Integration:** Snowflake seamlessly combines CSV data, audio transcripts, and PDF documents
2. **AI-Powered Insights:** Cortex AI functions extract meaning from unstructured text without manual coding
3. **Business Impact:** Data directly translates to actionable metrics (revenue at risk, churn probability)
4. **Scalability:** Same techniques work for 25 calls or 25,000 calls

### Next Steps

**1. Use Snowflake Intelligence Agent:**
Ask natural language questions like:
- "Which customers in Penang have the highest churn risk?"
- "What are the top network issues causing customer complaints?"
- "Show me revenue at risk from business customers"

**2. Build Production Solutions:**
- Create scheduled dashboards
- Set up alerts for at-risk customers
- Automate churn prediction scoring

**3. Extend the Analysis:**
- Add more data sources (social media, NPS surveys)
- Build advanced ML models
- Create real-time monitoring dashboards

**Thank you for completing the CelcomDigi Snowflake Intelligence Hands-On Lab!**